# **LoopsResultsAnalysis**

## **1. Importing and First Proccesing**

In [1]:
import pandas as pd
from DataCleaning import *
from ProcessingConfig import *

pd.options.display.max_columns = 50

In [5]:
raw_data = pd.read_excel(cleaning_config['raw_data_path'])
print(f'original shape: {raw_data.shape}')
raw_data = clean_data(raw_data)
print(f'final shape: {raw_data.shape}')

original shape: (13294, 32)
-- drop_first_loop: 1630 rows were filtered out.
-- drop_first_line: 2437 rows were filtered out.
-- filter_trail_outliers: 721 rows were filtered out.
-- filter_step_outliers: 242 rows were filtered out.
final shape: (8264, 26)


In [6]:
n_subjects = raw_data['subject'].nunique()
print(f'number of subjects: {n_subjects}')

n_sessions = raw_data[['subject', 'trial_set']].drop_duplicates().shape[0]
print(f'number of sessions: {n_sessions}')

number of subjects: 15
number of sessions: 29


## **2. Priming Effect Testing**

In [7]:
switching_diff = pd.DataFrame()

# mean response time and success rate, group by loop type and loop switching.
switching_diff['mean_response_time'] = raw_data[raw_data['correct']].groupby(['loop_type', 'loop_type_switch'])['rt'].mean()
switching_diff['mean_success_rate'] = raw_data.groupby(['loop_type', 'loop_type_switch'])['correct'].mean()

# loop ending response time, group by loop type and loop switching.
loop_end_mask = (raw_data['response_needed']) & (raw_data['expected_response'].isnull()) & (raw_data['correct'])
switching_diff['loop_end_mean_response_time'] = raw_data.loc[loop_end_mask].groupby(['loop_type', 'loop_type_switch'])['rt'].mean()

switching_diff

mean_response_time  mean_success_rate  \
loop_type loop_type_switch                                          
for       False                    1304.817951           0.977205   
          True                     1339.163618           0.981546   
while     False                    1567.088924           0.978626   
          True                     1574.004692           0.975585   

                            loop_end_mean_response_time  
loop_type loop_type_switch                               
for       False                              591.438721  
          True                               583.686941  
while     False                              813.898551  
          True                               788.949550